In [8]:
from os import walk
import os
import pandas as pd
import subprocess
import sys
import nltk
import time
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import pos_tag
import time
import json
import re
import urllib2
import unicodedata
from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize
import tqdm

wordnet_tags = ['n', 'v', 'a', 's', 'r']
%matplotlib inline

In [11]:
# from http://brandonrose.org/clustering



from nltk.stem import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
stemmer = SnowballStemmer("english")
lemmer = WordNetLemmatizer()
wordnet_tags = ['n', 'v', 'a', 's', 'r']

def tokenize_and_stem_n_lem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    ''' 
    Old code gave seperate lists for lem and stem; I lemmed, then just stemmed the result
    #stems = [stemmer.stem(t) for t in filtered_tokens]
    #lems = [lemmer.lemmatize(l) for l in filtered_tokens]
    '''
    stems = [stemmer.stem(lemmer.lemmatize(l)) for l in filtered_tokens]
    
    #return stems,lems
    return stems

def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [5]:
#strip any proper names from a text...unfortunately right now this is yanking the first word from a sentence too.
import string
def strip_proppers(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent) if word.islower()]
    return "".join([" "+i if not i.startswith("'") and i not in string.punctuation else i for i in tokens]).strip()

In [6]:
#strip any proper nouns (NNP) or plural proper nouns (NNPS) from a text
from nltk.tag import pos_tag

def strip_proppers_POS(text):
    tagged = pos_tag(text.split()) #use NLTK's part of speech tagger
    non_propernouns = [word for word,pos in tagged if pos != 'NNP' and pos != 'NNPS']
    return non_propernouns

In [16]:
path        = os.path.abspath(os.getcwd())
TESTDIR     = os.path.normpath(os.path.join(os.path.expanduser("~"),"projects","LC3-Creations", "examples","KDDsample"))

corpus = {}

for dirName, subdirList, fileList in walk(TESTDIR):    
    for fileName in fileList:
        if fileName.startswith('p') and fileName.endswith('.pdf'):
            a = unicode(subprocess.check_output(['pdf2txt.py',str(os.path.normpath(os.path.join(TESTDIR,fileName)))]),errors='ignore')
            document = unicodedata.normalize('NFKD', a).encode('ascii','ignore')

            if len(document)<300:
                pass
            else:

                # The entire document
                body = re.sub('[\s]'," ",document)

                # Getting title
                title = re.findall("^[^\\n\\n]+",document)[0]

                # Getting the abstract
                try:
                    abstract = re.findall (r'(Abstract|ABSTRACT)([^]]*)\n',document[:2000])
                except IndexError:
                    abstract = re.findall (r'(Abstract|ABSTRACT)([^]]*)\n',document[:2000])[0]
                else:
                    abstract = abstract = re.findall (r'(Abstract|ABSTRACT)([^]]*)\n',document[:2000])[0][1]

                if isinstance(abstract, tuple):
                    abstract = re.sub('[\s]'," ",abstract[1])
                elif isinstance(abstract,list):
                    abstract = re.sub('[\s]'," ",abstract[1])
                elif isinstance(abstract,str):
                    abstract = re.sub('[\s]'," ", abstract)


                # Extracts section with names and email addresses only
                section  = re.findall (r'\n\n([^]]*)\n\n(Abstract|ABSTRACT)',document[:2000])

                type(section[0][0])

                if isinstance(section, list):
                    section = re.sub('[\s]'," ",section[0][0])
                else:
                    section = re.sub('[\s]'," ",section)


                # Code to extract entities from top section of pdf and store a relationship tree
                tagged = nltk.pos_tag(nltk.word_tokenize(section))
                entities = nltk.chunk.ne_chunk(tagged)

                # Another entity extractor
                st = StanfordNERTagger('/Users/linwood/stanford-corenlp-full-2015-04-20/classifiers/english.conll.4class.distsim.crf.ser.gz',
                       '/Users/linwood/stanford-corenlp-full-2015-04-20/stanford-corenlp-3.5.2.jar',
                       encoding='utf-8')
                tokenized_text = word_tokenize(section)
                stanentities = st.tag(tokenized_text)

                # Calls function to lemmatize and stem the document; stores the result
                tokenize_and_stem_n_lem(abstract);

                '''
                This gives seperate lists for lem and stem; replacement code stores combin
                # Creates the json document format to store the files
                corpus[str(fileName)]={}
                corpus[str(fileName)]={'Title':title,'Abstract':abstract,'Entities':entities, 
                                       "Stanford ER":stanentities, "Stems": tokenize_and_stem_n_lem(abstract)[0], 
                                      "Lems": tokenize_and_stem_n_lem(abstract)[1]}'''

                # Creates the json document format to store the files
                corpus[str(fileName)]={}
                corpus[str(fileName)]={'Title':title,'body':body,'Abstract':abstract,'Entities':entities, 
                                       "Stanford ER":stanentities, "Stems": tokenize_and_stem_n_lem(abstract)}
  


CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.01 µs


CalledProcessError: Command '['pdf2txt.py', '/Users/linwood/projects/LC3-Creations/examples/KDDsample/p1.pdf']' returned non-zero exit status 1

In [18]:
# Creating lists for abstracts, titles, and entities

abstracts = [i['Abstract'] for i in corpus.values()[:]]
titles = [i['Title'] for i in corpus.values()[:]]
ents = [i['Entities'] for i in corpus.values()[:]]
bodies = [i['body'] for i in corpus.values()[:]]


totalvocab_stemmed = []
totalvocab_tokenized = []
for i in bodies:
    allwords_stemmed = tokenize_and_stem_n_lem(i) #for some reason, had to add index in list here
    totalvocab_stemmed.extend(allwords_stemmed) #extend the 'totalvocab_stemmed' list
    
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)

In [24]:
from sklearn.feature_extraction import stop_words
stopwords = set([l for l in stop_words.ENGLISH_STOP_WORDS])


In [25]:
from gensim import corpora, models, similarities 

#remove proper names
%time preprocess = [strip_proppers_POS(doc) for doc in bodies]

#tokenize
%time tokenized_text = [tokenize_and_stem_n_lem(text) for text in preprocess[0]]

#remove stop words
%time texts = [[word for word in text if word not in stopwords] for text in tokenized_text]

CPU times: user 3min 11s, sys: 1.71 s, total: 3min 13s
Wall time: 3min 12s
CPU times: user 2.07 s, sys: 96.9 ms, total: 2.16 s
Wall time: 2.1 s
CPU times: user 6.43 ms, sys: 35 µs, total: 6.47 ms
Wall time: 6.5 ms


In [26]:
#create a Gensim dictionary from the texts
dictionary = corpora.Dictionary(texts)

#remove extremes (similar to the min/max df step used when creating the tf-idf matrix)
dictionary.filter_extremes(no_below=1, no_above=0.8)

#convert the dictionary to a bag of words corpus for reference
corpus = [dictionary.doc2bow(text) for text in texts]

In [28]:
%time lda = models.LdaModel(corpus, num_topics=5, id2word=dictionary, update_every=5, chunksize=10000, passes=100)

CPU times: user 5min 36s, sys: 537 ms, total: 5min 37s
Wall time: 5min 37s


In [29]:
lda.show_topics()

[u'0.196*subgraph + 0.118*graph + 0.100*densiti + 0.065*g + 0.047*dens + 0.036*cid:48 + 0.032*-compact + 0.031*result + 0.014*greedi + 0.012*propos',
 u'0.156*g + 0.065*node + 0.044*comput + 0.034*region + 0.030*nd + 0.023*base + 0.022*repres + 0.022*return + 0.020*follow + 0.020*core',
 u'0.094*v + 0.035*compon + 0.031*contain + 0.028*line + 0.026*optim + 0.025*residu + 0.023*test + 0.022*identi + 0.021*becaus + 0.019*dataset',
 u'0.089*algorithm + 0.070*densest + 0.038*ani + 0.032*edg + 0.030*model + 0.029*connect + 0.027*report + 0.026*larg + 0.024*remov + 0.021*invalid',
 u'0.064*maxim + 0.058*use + 0.044*local + 0.031*prune + 0.027*time + 0.027*sub- + 0.027*denit + 0.024*relat + 0.018*k + 0.017*u']

In [32]:
lda.top_topics()

TypeError: top_topics() takes at least 2 arguments (1 given)

In [31]:
import numpy as np
topics_matrix = lda.show_topics(formatted=False, num_words=20)
topics_matrix = np.array(topics_matrix)

topic_words = topics_matrix[:,:,1]
for i in topic_words:
    print([str(word) for word in i])

['subgraph', 'graph', 'densiti', 'g', 'dens', 'cid:48', '-compact', 'result', 'greedi', 'propos', 'sinc', 'area', 'approach', 'problem', 'number', 'prove', 'set', 'delet', 'veric', 'differ']
['g', 'node', 'comput', 'region', 'nd', 'base', 'repres', 'return', 'follow', 'core', 'top-k', 'procedur', 'bound', 'lower', 'shown', 'origin', 'communiti', 'obtain', 'onli', 'case']
['v', 'compon', 'contain', 'line', 'optim', 'residu', 'test', 'identi', 'becaus', 'dataset', 'identifi', 'high', 'denot', 'sever', 'lemma', 'denser', 'n', '|e', 'tend', 'true']
['algorithm', 'densest', 'ani', 'edg', 'model', 'connect', 'report', 'larg', 'remov', 'invalid', 'network', 'properti', 'ha', 'size', 'larger', 'cid:100', 'cid:101', 'max', 'techniqu', 'con-']
['maxim', 'use', 'local', 'prune', 'time', 'sub-', 'denit', 'relat', 'k', 'u', 'dene', 'nding', 'compact', 'studi', 'doe', 'smaller', 'maximum', 'exist', '1/|v', 'iter']
